# Capstone Project
*Author: Samuel Leadley*
## Data Cleaning and EDA
## Table of Contents
1. [Load Data](#Load-Data)
2. [Data Cleaning](#Data-Cleaning)
3. [Export Data](#Export-Data)
4. [EDA](#EDA)

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import datetime
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import regex as re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

%matplotlib inline

## Load Data

In [2]:
shareholder_letters = pd.read_csv('../datasets/leadsheet.csv', encoding='Latin-1')
shareholder_letters.head()

,company,ticker,sector,year,letter_to_shareholders
0,Goldman Sachs,GS,Financials,1999,"This is our first letter to shareholders, incl..."
1,Goldman Sachs,GS,Financials,2000,2000 was a remarkable year for Goldman Sachs a...
2,Goldman Sachs,GS,Financials,2001,It is impossible to discuss 2001 without begin...
3,Goldman Sachs,GS,Financials,2002,It was a challenging year for Goldman Sachs. T...
4,Goldman Sachs,GS,Financials,2003,"Looking back on 2003, we take pride in our per..."


In [3]:
# check shape
shareholder_letters.shape

(166, 5)

## Data Cleaning
### Check for nulls

In [4]:
# check nulls for each column
shareholder_letters.isnull().sum()

company                   0
ticker                    0
sector                    0
year                      0
letter_to_shareholders    0
dtype: int64

### Check Data Types

In [5]:
shareholder_letters.dtypes

company                   object
ticker                    object
sector                    object
year                       int64
letter_to_shareholders    object
dtype: object

### Mapping Target Variable

In [6]:
ticker_lst = list(shareholder_letters['ticker'].unique())
ticker_lst

['GS', 'BAC', 'CVX', 'HAL', 'QCOM', 'IBM', 'ADBE', 'XRX', 'UHS', 'UNH', 'CVS']

In [7]:
# reading in net income data for each company
ni_data = pd.read_csv('../datasets/ni_data.csv')
ni_data.head()

,year,net_income,unit,ticker,target
0,2018,10.46B,B,GS,1.0
1,2017,4.286B,B,GS,0.0
2,2016,7.398B,B,GS,1.0
3,2015,6.083B,B,GS,0.0
4,2014,8.477B,B,GS,1.0


Net income data was collected from y-charts, an online financial data research company. The net income data came in either billions (denoted by a B) or millions (denoted by a M). Below I will convert all figures into billions. The target variable was calculated in excel by comparing the net income from the previous year to that of the current year. If the net income increase year-over-year it was labeled as a 1 and if it had decreased it was labeled as a 0 (there were no istances of neither an increase nor decrease). 

In [8]:
ni_data['net_income'] = [i.replace('B', '') for i in ni_data['net_income']]
ni_data['net_income'] = [i.replace('M', '') for i in ni_data['net_income']]
ni_data['net_income'] = ni_data['net_income'].astype(float)

In [9]:
def net_income_to_B(df):
    for row,value in enumerate(df['net_income']):
        if df['unit'][row] == 'B':
            df['net_income'][row] = value * 1
        else:
            df['net_income'][row] = value * 0.001
    return(df)

In [10]:
ni_data = net_income_to_B(ni_data)

/Users/samuelleadley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/samuelleadley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
ni_data.head()

,year,net_income,unit,ticker,target
0,2018,10.460,B,GS,1.0
1,2017,4.286,B,GS,0.0
2,2016,7.398,B,GS,1.0
3,2015,6.083,B,GS,0.0
4,2014,8.477,B,GS,1.0


In [12]:
shareholder_letters.head()

,company,ticker,sector,year,letter_to_shareholders
0,Goldman Sachs,GS,Financials,1999,"This is our first letter to shareholders, incl..."
1,Goldman Sachs,GS,Financials,2000,2000 was a remarkable year for Goldman Sachs a...
2,Goldman Sachs,GS,Financials,2001,It is impossible to discuss 2001 without begin...
3,Goldman Sachs,GS,Financials,2002,It was a challenging year for Goldman Sachs. T...
4,Goldman Sachs,GS,Financials,2003,"Looking back on 2003, we take pride in our per..."


In [13]:
ni_data['index'] = ni_data['ticker'] + ni_data['year'].astype(str)
shareholder_letters['index'] = shareholder_letters['ticker'] + shareholder_letters['year'].astype(str)

In [14]:
shareholder_letters = pd.merge(shareholder_letters, ni_data, on='index')
shareholder_letters.drop(['index', 'year_y', 'ticker_y', 'unit'], axis = 1, inplace=True)
shareholder_letters.columns = ['company', 'ticker', 'sector', 'year', 'letter_to_shareholder', 'net_income', 'target']

In [15]:
shareholder_letters.head()

,company,ticker,sector,year,letter_to_shareholder,net_income,target
0,Goldman Sachs,GS,Financials,1999,"This is our first letter to shareholders, incl...",2.708,NaN
1,Goldman Sachs,GS,Financials,2000,2000 was a remarkable year for Goldman Sachs a...,3.067,1.0
2,Goldman Sachs,GS,Financials,2001,It is impossible to discuss 2001 without begin...,2.310,0.0
3,Goldman Sachs,GS,Financials,2002,It was a challenging year for Goldman Sachs. T...,2.114,0.0
4,Goldman Sachs,GS,Financials,2003,"Looking back on 2003, we take pride in our per...",3.005,1.0


In [16]:
shareholder_letters.isnull().sum()

company                  0
ticker                   0
sector                   0
year                     0
letter_to_shareholder    0
net_income               0
target                   1
dtype: int64

In [17]:
shareholder_letters[shareholder_letters['target'].isnull() == True]

,company,ticker,sector,year,letter_to_shareholder,net_income,target
0,Goldman Sachs,GS,Financials,1999,"This is our first letter to shareholders, incl...",2.708,NaN


In [19]:
# imputing the first Goldman Sachs letter to shareholders as positive after manually reading it
shareholder_letters['target'][0] = 1

/Users/samuelleadley/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Goldman Sachs IPO'd in 1999 so y-charts did not have any net income data prior to 1999. Thus I could not compare the net income year-over-year. Instead I read the letter to shareholders and dermined it was positive. 

In [20]:
shareholder_letters.head()

,company,ticker,sector,year,letter_to_shareholder,net_income,target
0,Goldman Sachs,GS,Financials,1999,"This is our first letter to shareholders, incl...",2.708,1.0
1,Goldman Sachs,GS,Financials,2000,2000 was a remarkable year for Goldman Sachs a...,3.067,1.0
2,Goldman Sachs,GS,Financials,2001,It is impossible to discuss 2001 without begin...,2.310,0.0
3,Goldman Sachs,GS,Financials,2002,It was a challenging year for Goldman Sachs. T...,2.114,0.0
4,Goldman Sachs,GS,Financials,2003,"Looking back on 2003, we take pride in our per...",3.005,1.0


In [ ]:
# read in each of the companies share price history
GS = pd.read_csv('../datasets/GS_stock_data.csv')
BAC = pd.read_csv('../datasets/BAC_stock_data.csv')
CVX = pd.read_csv('../datasets/CVX_stock_data.csv')
HAL = pd.read_csv('../datasets/HAL_stock_data.csv')
QCOM = pd.read_csv('../datasets/QCOM_stock_data.csv')
IBM = pd.read_csv('../datasets/IBM_stock_data.csv')
ADBE = pd.read_csv('../datasets/ADBE_stock_data.csv')
XRX = pd.read_csv('../datasets/XRX_stock_data.csv')
UHS = pd.read_csv('../datasets/UHS_stock_data.csv')
UNH = pd.read_csv('../datasets/UNH_stock_data.csv')
CVS = pd.read_csv('../datasets/CVS_stock_data.csv')

In [ ]:
# create a dictionary of all my dataframes 
df_dict = {'GS': GS,
          'BAC': BAC,
          'CVX': CVX,
          'HAL': HAL,
          'QCOM': QCOM,
          'IBM': IBM,
          'ADBE': ADBE,
          'XRX': XRX,
          'UHS': UHS,
          'UNH': UNH,
          'CVS': CVS}

In [ ]:
# test dictionary
df_dict['BAC'].head()

In [ ]:
# define a function that returns a binarized target variable based on if the tickers stock had increased within a year
def get_first_last_price_df(df_temp):
    df = df_temp.copy()
    df.drop(['open', 'high', 'low', 'volume'], axis=1, inplace=True)
    df['date1'] = df['date']
    df['date1'] = pd.to_datetime(df['date1'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace = True)
    df_first = df.resample('A').agg({'date1': 'min', 'close': 'first'})
    df_last = df.resample('A').agg({'date1': 'max', 'close' : 'last'})
    df = pd.concat([df_first, df_last], axis = 1)
    df.columns = ['first_date', 'first_close', 'last_date', 'last_close']
    df.reset_index(inplace=True)
    df['year'] = df['date'].dt.year
    df['target'] = df['last_close'] - df['first_close']
    df['target'] = (df['target'] > 0).astype(int)
    df.drop(['date', 'first_date', 'first_close', 'last_date', 'last_close'], axis = 1, inplace = True)
    return df

In [ ]:
# apply function to all dataframes
for ticker in df_dict.keys():
    df_dict[ticker] = get_first_last_price_df(df_dict[ticker])

In [ ]:
# identify which ticker belongs to which dataframe
df_dict['GS']['ticker'] = 'GS'
df_dict['BAC']['ticker'] = 'BAC'
df_dict['CVX']['ticker'] = 'CVX'
df_dict['HAL']['ticker'] = 'HAL'
df_dict['QCOM']['ticker'] = 'QCOM'
df_dict['IBM']['ticker'] = 'IBM'
df_dict['ADBE']['ticker'] = 'ADBE'
df_dict['XRX']['ticker'] = 'XRX'
df_dict['UHS']['ticker'] = 'UHS'
df_dict['UNH']['ticker'] = 'UNH'
df_dict['CVS']['ticker'] = 'CVS'

In [ ]:
# combine the dataframes
target_df = pd.concat([df_dict['GS'], df_dict['BAC'], 
                       df_dict['CVX'], df_dict['HAL'], 
                       df_dict['QCOM'], df_dict['IBM'],
                       df_dict['ADBE'], df_dict['XRX'],
                       df_dict['UHS'], df_dict['UNH'],
                       df_dict['CVS']])
target_df.reset_index(inplace=True)
target_df.drop('index', axis = 1, inplace = True)
# create new index to merge the shareholder_letters with target_df
target_df['index'] = target_df['ticker'] + target_df['year'].astype(str)

In [ ]:
# create new index to merge the shareholder_letters with target_df
shareholder_letters['index'] = shareholder_letters['ticker'] + shareholder_letters['year'].astype(str)

In [ ]:
# merge the shareholder_letters with target_df
shareholder_letters = pd.merge(shareholder_letters, target_df, on='index')
shareholder_letters.drop(['index', 'year_y', 'ticker_y'], axis = 1, inplace=True)
shareholder_letters.columns = ['company', 'ticker', 'sector', 'year', 'letter_to_shareholder', 'target']

In [ ]:
shareholder_letters.head()

### Clean Text

In [ ]:
shareholder_letters['letter_to_shareholder'] = shareholder_letters['letter_to_shareholder'].astype(str)

In [ ]:
shareholder_letters['letter_to_shareholder'] = shareholder_letters['letter_to_shareholder'].apply(lambda x: re.sub('[^ a-zA-Z!#]','', x).lower())


In [ ]:
shareholder_letters

In [ ]:
# check the balance of target
shareholder_letters['target'].value_counts(normalize=True)

## Export Data

In [ ]:
shareholder_letters.to_csv('../datasets/clean_df_1.csv')

## EDA
### CountVectorizer with Entire Corpus

In [ ]:
# Instantiate CountVectorizer to see which words are most common in the letters to shareholders
# Removed stop words to see the more unique words that appear
cvec = CountVectorizer(stop_words='english')
# Fit CountVectorizer to title
cvec.fit(shareholder_letters['letter_to_shareholder'])

In [ ]:
# Transform the corpus
freq = cvec.transform(shareholder_letters['letter_to_shareholder'])
# Convert into a DataFrame
freq_df = pd.DataFrame(freq.toarray(), columns = cvec.get_feature_names())

In [ ]:
freq_df.shape

In [ ]:
# Looking at the initial list of most common words in the letters to shareholders
freq_df.mean().sort_values(ascending=False)[0:21]

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
freq_df.mean().sort_values(ascending=False)[0:21].plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Words', size = 15)
plt.title('Most Common Words in Letters to Shareholders', size = 15);

The above graph shows the most common words in the letters to shareholders. It is not surprising that words like business, percent, and year are some of the most common words.

### CountVectorizer with Positive Class

In [ ]:
positive_class = shareholder_letters[shareholder_letters['target'] == 1][['letter_to_shareholder']]

In [ ]:
# Instantiate CountVectorizer to see which words are most common in the positive letters to shareholders
# Removed stop words to see the more unique words that appear
cvec = CountVectorizer(stop_words='english')
# Fit CountVectorizer to title
cvec.fit(positive_class['letter_to_shareholder'])

In [ ]:
# Transform the corpus
pos_freq = cvec.transform(positive_class['letter_to_shareholder'])
# Convert into a DataFrame
pos_freq_df = pd.DataFrame(pos_freq.toarray(), columns = cvec.get_feature_names())

In [ ]:
# Looking at the initial list of most common words in the positive letters to shareholders
pos_freq_df.mean().sort_values(ascending=False)[0:21]

In [ ]:
# create a dataframe of the positive words to later compare to the top words in the negative class
positive_words = pd.DataFrame(pos_freq_df.mean().sort_values(ascending=False))
positive_words.reset_index(inplace=True)

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
pos_freq_df.mean().sort_values(ascending=False)[0:21].plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Words', size = 15)
plt.title('Most Common Words in the Positive Letters to Shareholders', size = 15);

The words are very similar to that of the whole corpus with the first four words exactly the same. This may indicate that using TfIdf may be a better vectorizer to use because it will penalize words that are common within the corpus. 

### CountVectorizer with Negative Class

In [ ]:
negative_class = shareholder_letters[shareholder_letters['target'] == 0][['letter_to_shareholder']]

In [ ]:
# Instantiate CountVectorizer to see which words are most common in the negative letters to shareholders
# Removed stop words to see the more unique words that appear
cvec = CountVectorizer(stop_words='english')
# Fit CountVectorizer to title
cvec.fit(negative_class['letter_to_shareholder'])

In [ ]:
# Transform the corpus
neg_freq = cvec.transform(negative_class['letter_to_shareholder'])
# Convert into a DataFrame
neg_freq_df = pd.DataFrame(neg_freq.toarray(), columns = cvec.get_feature_names())

In [ ]:
neg_freq_df.shape

In [ ]:
# Looking at the initial list of most common words in the negative letters to shareholders
neg_freq_df.mean().sort_values(ascending=False)[0:21]

In [ ]:
# create a dataframe of the negative words to later compare to the top words in the negative class
negative_words = pd.DataFrame(neg_freq_df.mean().sort_values(ascending=False))
negative_words.reset_index(inplace=True)

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
neg_freq_df.mean().sort_values(ascending=False)[0:21].plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Words', size = 15)
plt.title('Most Common Words in the Negative Letters to Shareholders', size = 15);

Both the positive and negative letters to shareholders have very similar words. It is logical that most of these words would come up in both classes because the letters are written using very similar jargon and discourse.

### Positve vs. Negative Words

In [ ]:
set(positive_words['index']) - set(negative_words['index']) 

In [ ]:
set(negative_words['index']) - set(positive_words['index'])

### CountVectorizer with Entire Corpus: ngram_range = (2, 2)

In [ ]:
# Instantiate CountVectorizer to see which words are most common in the letters to shareholders
# Removed stop words to see the more unique words that appear
cvec = CountVectorizer(stop_words='english', ngram_range=(2,2))
# Fit CountVectorizer to title
cvec.fit(shareholder_letters['letter_to_shareholder'])

In [ ]:
# Transform the corpus
freq = cvec.transform(shareholder_letters['letter_to_shareholder'])
# Convert into a DataFrame
freq_df = pd.DataFrame(freq.toarray(), columns = cvec.get_feature_names())

In [ ]:
freq_df.columns

In [ ]:
# Looking at the initial list of most common words in the letters to shareholders
freq_df.mean().sort_values(ascending=False)[0:21]

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
freq_df.mean().sort_values(ascending=False)[0:21].plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Words', size = 15)
plt.title('Most Common Word Pairs in the Letters to Shareholders', size = 15);

### CountVectorizer with Positive Class: ngram_range = (2, 2)

In [ ]:
# Instantiate CountVectorizer to see which word pairs are most common in the positive letters to shareholders
# Removed stop words to see the more unique words that appear
cvec = CountVectorizer(stop_words='english', ngram_range=(2,2))
# Fit CountVectorizer to title
cvec.fit(positive_class['letter_to_shareholder'])

In [ ]:
# Transform the corpus
pos_freq = cvec.transform(positive_class['letter_to_shareholder'])
# Convert into a DataFrame
pos_freq_df = pd.DataFrame(pos_freq.toarray(), columns = cvec.get_feature_names())

In [ ]:
# Looking at the initial list of most common word pairs in the positive letters to shareholders
pos_freq_df.mean().sort_values(ascending=False)[0:21]

In [ ]:
# Visualization of most frequent word pairs in the letters to shareholders
plt.figure(figsize = (12, 10))
pos_freq_df.mean().sort_values(ascending=False)[0:21].plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Words', size = 15)
plt.title('Most Common Word Pairs in the Positive Letters to Shareholders', size = 15);

### CountVectorizer with Negative Class: ngram_range(2, 2)

In [ ]:
# Instantiate CountVectorizer to see which word pairs are most common in the negative letters to shareholders
# Removed stop words to see the more unique words that appear
cvec = CountVectorizer(stop_words='english', ngram_range=(2,2))
# Fit CountVectorizer to title
cvec.fit(negative_class['letter_to_shareholder'])

In [ ]:
# Transform the corpus
neg_freq = cvec.transform(negative_class['letter_to_shareholder'])
# Convert into a DataFrame
neg_freq_df = pd.DataFrame(neg_freq.toarray(), columns = cvec.get_feature_names())

In [ ]:
# Looking at the initial list of most common words in the positive letters to shareholders
neg_freq_df.mean().sort_values(ascending=False)[0:21]

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
neg_freq_df.mean().sort_values(ascending=False)[0:21].plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Words', size = 15)
plt.title('Most Common Word Pairs in the Negative Letters to Shareholders', size = 15);

### Industry EDA

In [ ]:
ls = []
for letter in shareholder_letters['letter_to_shareholder']:
    length = len(letter)
    ls.append(length)
    letter_len = pd.DataFrame(ls)

In [ ]:
shareholder_letters = pd.concat([shareholder_letters, letter_len], axis = 1)
shareholder_letters.rename(columns={0: 'length'}, inplace=True)

In [ ]:
avg_sector_lengths = shareholder_letters.groupby('sector').mean()[['length']]

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
avg_sector_lengths.sort_values(by = 'length', ascending=False).plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Length', size = 15)
plt.ylabel('Sector', size = 15)
plt.title('Average Length of Letters to Shareholders Across Sectors', size = 15);

As evidenced by tha graph above there are clear differences in the length of shareholders letters in each sector. The finacials sector has quite verbose letters to its shareholders while the energy sector like to keep its letters fairly short.

In [ ]:
postive_vs_negative_years = shareholder_letters.groupby('sector').mean()[['target']]

In [ ]:
# Visualization of most frequent words in the letters to shareholders
plt.figure(figsize = (12, 10))
postive_vs_negative_years.sort_values(by = 'target', ascending=False).plot(kind = 'barh', color='cornflowerblue')

plt.xlabel('Frequency', size = 15)
plt.ylabel('Sector', size = 15)
plt.title('Percentage of Positive vs. Negative Years by Sector', size = 15);

Looking at the graph above we can infer that the Heathcare sector fared the best over the time period having positive years more than 70 percent of the time while the Info Tech sector did the worst. Info Tech likley perfromed worse than the other sectors because of the dot com bubble however I am surpised that the financials sector fared second best seeing as it was also hit hard by the 2008 recession. 